In [27]:
import cv2 as cv
from ultralytics import YOLO
import os

In [3]:
vehicle_model=YOLO('yolov8n.pt')
liscence_model = YOLO('best.pt')

# Registation plate number recognition

In [33]:
vid="videos/vid1.mp4" 
video=cv.VideoCapture(vid)
ret = True
frame_number = -1
# all vehicle class IDs from the COCO dataset (car, motorbike, truck) https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
vehicles = [2,3,5,7] #classes from COCO datasets

#NOTE: remove all files in 'plates' folder before running this script
#to be used only if you want to remove all files in 'plates' folder
# if os.path.exists('plates'):
#     for file in os.listdir('plates'):
#         os.remove(os.path.join('plates', file))
        
# get frame rate
frame_rate = 1

while ret:
    # effectively skip frames
    video.set(cv.CAP_PROP_POS_MSEC, frame_number*1000*frame_rate)
    ret, frame = video.read()
    
    # break if no frame
    if not ret:
        break
    
    frame_number += 1
    
    detections = vehicle_model.track(frame, persist=True, save=True)[0]
    for detection in detections.boxes.data.tolist():
        
        #This is to prevent errors when the model does not return class_id
        if len(detection) == 6:
            x1, y1, x2, y2, track_id, score = detection
            class_id = 0 # or any default value
        else:
            x1, y1, x2, y2, track_id, score, class_id = detection
        
        if int(class_id) in vehicles and score > 0.5:
            vehicle_bounding_boxes = []
            vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
            for bbox in vehicle_bounding_boxes:
                
                roi = frame[int(y1):int(y2), int(x1):int(x2)]

                print(bbox)
                print(f"frame: {frame_number}")
                # license plate detector for region of interest
                license_plates = liscence_model(roi)[0]
                # print(license_plates)
                # check every bounding box for a license plate
                for license_plate in license_plates.boxes.data.tolist():
                    plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                    # verify detections
                    print(license_plate, 'track_id: ' + str(bbox[4]))
                    plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                    # save license plate in folder called 'plates'
                    if not os.path.exists('plates'):
                        os.makedirs('plates')
                    cv.imwrite(str(f'plates/plate_{track_id}.jpg'), plate)
                    #cv.imwrite(str(track_id) + '.jpg', plate)

video.release()


0: 416x640 1 car, 120.8ms
Speed: 3.7ms preprocess, 120.8ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)
Results saved to /home/kelmith/Kelmith/Projects/parking/parkingvision/runs/detect/predict2
[97.86946105957031, 179.06080627441406, 996.3622436523438, 644.4365234375, 243.0, 0.8898075222969055]
frame: 0

0: 352x640 1 licence, 388.0ms
Speed: 2.0ms preprocess, 388.0ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)
[504.7481689453125, 351.8430480957031, 683.5484008789062, 399.04736328125, 0.9147810339927673, 0.0] track_id: 243.0

0: 416x640 2 cars, 112.0ms
Speed: 5.6ms preprocess, 112.0ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)
Results saved to /home/kelmith/Kelmith/Projects/parking/parkingvision/runs/detect/predict2
[88.50310516357422, 215.37017822265625, 973.5576171875, 681.8275756835938, 243.0, 0.8898075222969055]
frame: 1

0: 352x640 1 licence, 500.8ms
Speed: 3.1ms preprocess, 500.8ms inference, 1.6ms postprocess per ima